In [8]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[4]") \
   .appName("tc") \
   .enableHiveSupport() \
   .getOrCreate()

sc = spark.sparkContext

In [9]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

# print(string.punctuation)
def mytoken_lemm(text):
    tokens = word_tokenize(text)
    tokens = [w.lower() for w in tokens]
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in stripped if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    lemms = []
    for item in words:
        lemms.append(WordNetLemmatizer().lemmatize(item))
    return lemms
#     full = []
#     full.append(lemms)
#     return full

# def mytoken_stem(text):
#     tokens = word_tokenize(text)
#     tokens = [w.lower() for w in tokens]
#     table = str.maketrans('', '', string.punctuation)
#     stripped = [w.translate(table) for w in tokens]
#     words = [word for word in stripped if word.isalpha()]
#     stop_words = set(stopwords.words('english'))
#     words = [w for w in words if not w in stop_words]
#     stems = []
#     for item in words:
#         stems.append(PorterStemmer().stem(item))
#     return stems

In [10]:
# Remove first row
import csv
trainRDD = sc.textFile('data/train.csv')
trainHeader = trainRDD.first()
print(trainHeader)
trainRawRDD = trainRDD.filter(lambda line: line != trainHeader).mapPartitions(lambda x: csv.reader(x)) #csv.reader return a list 
trainRawRDD.take(2)

"id","comment_text","toxic","severe_toxic","obscene","threat","insult","identity_hate"


[['0000997932d777bf',
  "ExplanationWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",
  '0',
  '0',
  '0',
  '0',
  '0',
  '0'],
 ['000103f0d9cfb60f',
  "D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)",
  '0',
  '0',
  '0',
  '0',
  '0',
  '0']]

In [11]:
# Data Transformations(tokenize)
trainRDD = trainRawRDD.map(lambda line: mytoken_lemm(line[1])+line[2:8])
trainRDD.take(2)

[['explanationwhy',
  'edits',
  'made',
  'username',
  'hardcore',
  'metallica',
  'fan',
  'reverted',
  'nt',
  'vandalism',
  'closure',
  'gas',
  'voted',
  'new',
  'york',
  'doll',
  'fac',
  'please',
  'nt',
  'remove',
  'template',
  'talk',
  'page',
  'since',
  'retired',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0'],
 ['daww',
  'match',
  'background',
  'colour',
  'seemingly',
  'stuck',
  'thanks',
  'talk',
  'january',
  'utc',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0']]

In [12]:
def sep(line):
    comment = []
    full = []
    for word in line:  
        if word != '0':
            comment.append(word)
        else:
            full.append(word)
    full.append(comment)
    return full

trainRDD1 = trainRDD.map(lambda line: sep(line))
trainRDD1.take(2)


[['0',
  '0',
  '0',
  '0',
  '0',
  '0',
  ['explanationwhy',
   'edits',
   'made',
   'username',
   'hardcore',
   'metallica',
   'fan',
   'reverted',
   'nt',
   'vandalism',
   'closure',
   'gas',
   'voted',
   'new',
   'york',
   'doll',
   'fac',
   'please',
   'nt',
   'remove',
   'template',
   'talk',
   'page',
   'since',
   'retired']],
 ['0',
  '0',
  '0',
  '0',
  '0',
  '0',
  ['daww',
   'match',
   'background',
   'colour',
   'seemingly',
   'stuck',
   'thanks',
   'talk',
   'january',
   'utc']]]

In [13]:
# Transfer RDD to DataFrame
# trainDF = spark.createDataFrame(trainRDD1).toDF("comment_text","toxic","severe_toxic","obscene","threat","insult","identity_hate")

haha = spark.createDataFrame(trainRDD1).toDF("toxic","severe_toxic","obscene","threat","insult","identity_hate","comment_text")

haha.dtypes
# trainDF.show(1)

AnalysisException: 'java.lang.RuntimeException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient;'

In [137]:
trainDF.printSchema()

root
 |-- toxic: string (nullable = true)
 |-- severe_toxic: string (nullable = true)
 |-- obscene: string (nullable = true)
 |-- threat: string (nullable = true)
 |-- insult: string (nullable = true)
 |-- identity_hate: string (nullable = true)
 |-- comment_text: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [138]:
trainDF.dtypes

[('toxic', 'string'),
 ('severe_toxic', 'string'),
 ('obscene', 'string'),
 ('threat', 'string'),
 ('insult', 'string'),
 ('identity_hate', 'string'),
 ('comment_text', 'array<string>')]

In [139]:
trainDF.columns

['toxic',
 'severe_toxic',
 'obscene',
 'threat',
 'insult',
 'identity_hate',
 'comment_text']

In [144]:
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors,DenseVector
from pyspark.ml.feature import HashingTF,IDF,Tokenizer,RegexTokenizer,VectorAssembler

hashingTF = HashingTF(inputCol="comment_text", outputCol="rawCommentText", numFeatures=262144)
featurizedData = hashingTF.transform(trainDF)
featurizedData.dtypes

[('toxic', 'string'),
 ('severe_toxic', 'string'),
 ('obscene', 'string'),
 ('threat', 'string'),
 ('insult', 'string'),
 ('identity_hate', 'string'),
 ('comment_text', 'array<string>'),
 ('rawCommentText', 'vector')]

In [145]:
idf = IDF(inputCol="rawCommentText", outputCol="CommentText")
idfModel = idf.fit(featurizedData)

finalTrainDF = idfModel.transform(featurizedData)
finalTrainDF.dtypes

Py4JJavaError: An error occurred while calling o2933.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 140.0 failed 1 times, most recent failure: Lost task 1.0 in stage 140.0 (TID 393, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 253, in main
    process()
  File "/usr/local/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 248, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 379, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/util.py", line 55, in wrapper
    return f(*args, **kwargs)
  File "<ipython-input-134-b5

In [77]:
# Transfer RDD to DataFrame
# trainDF = spark.createDataFrame(trainRDD).toDF("comment_text","toxic","severe_toxic","obscene","threat","insult","identity_hate")
trainDF = spark.read.csv('data/train.csv',header=True)
trainDF.columns


['id',
 'comment_text',
 'toxic',
 'severe_toxic',
 'obscene',
 'threat',
 'insult',
 'identity_hate']

In [78]:
trainDF = trainDF.select([column for column in trainDF.columns if column !='id'])
trainDF.printSchema()

root
 |-- comment_text: string (nullable = true)
 |-- toxic: string (nullable = true)
 |-- severe_toxic: string (nullable = true)
 |-- obscene: string (nullable = true)
 |-- threat: string (nullable = true)
 |-- insult: string (nullable = true)
 |-- identity_hate: string (nullable = true)



In [79]:
# from pyspark.sql.functions import col
# trainDF.groupBy("toxic") \
#     .count() \
#     .orderBy(col("count").desc()) \
#     .show()

In [83]:
# actualDF = trainDF.withColumn(
#   "comment_text",
#   mytoken_lemm(trainDF.comment_text)
# )
# actualDF.show(1)

In [16]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression

# regular expression tokenizer
regexTokenizer = RegexTokenizer(inputCol="comment_text", outputCol="words", pattern="\\W")

# stop words
stop_words = list(stopwords.words('english'))
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(stop_words)

# bag of words count
countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=10000, minDF=5)

In [17]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
label_stringIdx = StringIndexer(inputCol = "toxic", outputCol = "label")

In [19]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, label_stringIdx])

# Fit the pipeline to training documents.
pipelineFit = pipeline.fit(trainDF)
dataset = pipelineFit.transform(trainDF)
dataset.show(5)


Py4JJavaError: An error occurred while calling o124.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 8.0 failed 1 times, most recent failure: Lost task 2.0 in stage 8.0 (TID 212, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$createTransformFunc$2: (string) => array<string>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$10$$anon$1.hasNext(WholeStageCodegenExec.scala:614)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$12.hasNext(Iterator.scala:439)
	at org.apache.spa